# Mapping Covid
## Import Necessary Libraries

In [6]:
import os
import numpy as np
import geopandas as gpd

## Process the Mapping Data
This section includes:
    * Loading the GeoJSON of FIPS Counties from Plotly
    * Creating dissolved GeoJSON areas for New York City, Kansas City and Joplin as outlined in the NYT Covid-19 Dataset

In [46]:
# read plotly GeoJSON file into geopandas dataframe
url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
gdf = gpd.read_file(url)

In [47]:
gdf.head()

,id,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,01001,0500000US01001,01,001,Autauga,County,594.436,"POLYGON ((-86.49677 32.34444, -86.71790 32.402..."
1,01009,0500000US01009,01,009,Blount,County,644.776,"POLYGON ((-86.57780 33.76532, -86.75914 33.840..."
2,01017,0500000US01017,01,017,Chambers,County,596.531,"POLYGON ((-85.18413 32.87053, -85.12342 32.772..."
3,01021,0500000US01021,01,021,Chilton,County,692.854,"POLYGON ((-86.51734 33.02057, -86.51596 32.929..."
4,01033,0500000US01033,01,033,Colbert,County,592.619,"POLYGON ((-88.13999 34.58170, -88.13925 34.587..."


### Create pseudo FIPS codes for merged fips regions
In the NYT Covid dataset, multiple FIPS regions are combined for New York City, Kansas City and Joplin. To accomodate this we generate psuedo FIPS codes for these regions based off the standard algorithim of 2 digit state code + a 3 digit county code.  Using the work by Jed Kolko as a model and for easy data merging, we use the following mappings:
* New York City = 36 + 991 (From official FIPS: 36005, 36061, 36047, 36081, 36085)
* Kansas City = 29 + 991 (From official FIPS: 29037, 29047, 29095, 29165)
* Joplin = 29 + 992 (From official FIPS: 29097, 29145)

In [48]:
# List of City FIPS
nyc = ['36005', '36061', '36047', '36081', '36085']
kc = ['29037', '29047', '29095', '29165']
joplin = ['29097', '29145']
cities = nyc + kc + joplin

# Subset geopandas datarfame to just these cities
gdf_cities = gdf[gdf['id'].isin(cities)].copy()

In [49]:
# Replace values of id, COUNTY, NAME columns with city designations
gdf_cities.loc[gdf_cities['id'].isin(nyc),['NAME', 'id', 'COUNTY']] = 'New York City', '36991', '991'
gdf_cities.loc[gdf_cities['id'].isin(kc),['NAME','id','COUNTY']] = 'Kansas City', '29991', '991'
gdf_cities.loc[gdf_cities['id'].isin(joplin),['NAME','id', 'COUNTY']] = 'Joplin', '29992', '992'

In [50]:
# Remove GEO_ID  and LSAD columns, and dissolve on new FIPS code while aggregating CENSUSAREA by summation
gdf_cities = gdf_cities[['id','STATE','COUNTY','NAME','CENSUSAREA', 'geometry']]
gdf_cities = gdf_cities.dissolve(by=['id','STATE','COUNTY','NAME'], aggfunc = 'sum')

In [51]:
# Remove indexing # and add NA GEO_ID and LSAD columns back in for concatenation with original dataframe 
gdf_cities.reset_index(inplace=True)
gdf_cities["GEO_ID"] = np.nan
gdf_cities["LSAD"] = np.nan
gdf_cities

,id,STATE,COUNTY,NAME,geometry,CENSUSAREA,GEO_ID,LSAD
0,29991,29,991,Kansas City,"POLYGON ((-94.11910 38.83317, -94.11966 38.568...",2118.786,NaN,NaN
1,29992,29,992,Joplin,"POLYGON ((-94.06890 36.74782, -94.61831 36.766...",1263.252,NaN,NaN
2,36991,36,991,New York City,"MULTIPOLYGON (((-74.14443 40.53516, -74.14870 ...",302.643,NaN,NaN


In [52]:
# Index both dataframes on the id (aka FIPS) column
gdf = gdf.set_index('id')
gdf_cities = gdf_cities.set_index('id')

In [53]:
# Select gdf_cities columns in same order as gdf for smooth appending
gdf_cities = gdf_cities[['GEO_ID','STATE','COUNTY','NAME','LSAD','CENSUSAREA','geometry']]

In [54]:
# Append the City Specific FIPS columns back into the main geopandas dataframe
NYT_fips = gdf.append(gdf_cities) 

In [55]:
NYT_fips.tail()

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
id,,,,,,,
51037,0500000US51037,51,037,Charlotte,County,475.271,"POLYGON ((-78.44332 37.07940, -78.49303 36.891..."
51041,0500000US51041,51,041,Chesterfield,County,423.297,"POLYGON ((-77.85180 37.35487, -77.85515 37.418..."
29991,NaN,29,991,Kansas City,NaN,2118.786,"POLYGON ((-94.11910 38.83317, -94.11966 38.568..."
29992,NaN,29,992,Joplin,NaN,1263.252,"POLYGON ((-94.06890 36.74782, -94.61831 36.766..."
36991,NaN,36,991,New York City,NaN,302.643,"MULTIPOLYGON (((-74.14443 40.53516, -74.14870 ..."


In [57]:
# Export merged GeoJSON dataframe to your local computer for later use. 
# This datafile is also located on github at 
NYT_fips.to_file("NYT_fips.json", driver="GeoJSON")